In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
import csv

In [ ]:
## Function that loads from a csv file the DualABMag minijpas catalog with associated photoz,
## odds, and GAIA apparent move data.
def load_cat_photoz_gaia(filename):
    with open(filename, mode='r') as csvfile:
        rdlns = csv.reader(csvfile, delimiter=',')
        next(rdlns, None)
        next(rdlns, None)
        
        number = []
        mag = []
        mag_err = []
        flags = []
        mflags = []
        photoz = []
        odds = []
        parallax = []
        parallax_err = []
        pmra = []
        pmra_err = []
        pmdec = []
        pmdec_err = []
        
        for line in rdlns:
            number.append(line[0])
            mag.append(line[1].split())
            mag_err.append(line[2].split())
            flags.append(line[3].split())
            mflags.append(line[4].split())
            photoz.append(line[5])
            odds.append(line[6])
            parallax.append(line[7])
            parallax_err.append(line[8])
            pmra.append(line[9])
            pmra_err.append(line[10])
            pmdec.append(line[11])
            pmdec_err.append(line[12])
            
    columns = [
        number, mag, mag_err,
        flags, mflags, photoz,
        odds, parallax, parallax_err,
        pmra, pmra_err, pmdec,
        pmdec_err
    ]
    cat_keys = [
        'number', 'mag', 'mag_err', 'flags',
        'mflags', 'photoz', 'odds', 'parallax',
        'parallax_err', 'pmra', 'pmra_err',
        'pmdec', 'pmdec_err'
    ]
    cat_types = [
        int, float, float, int,
        int, float, float, float,
        float, float, float, float, float
    ]
    cat = {}
    
    for col,key in zip(columns, cat_keys):
        cat[key] = np.array(col)
    # Substitute empty values by NumPy NaN
    for k,t in zip(cat.keys(), cat_types):
        cat[k][np.where(cat[k] == '')] = np.nan
        cat[k] = cat[k].astype(t)
    
    return cat

In [ ]:
## Function to compute the probability of a reliable excess for nb_ind and bb_ind
def p_lya_bbnb(bb_ind, nb_ind, catalog):
    bb_m_zero = cat['mag'][:, bb_ind]
    nb_m_zero = cat['mag'][:, nb_ind]
    odds = cat['odds']
    
    # Mask sources with negative flux values (i.e. m = 99)
    mask_neg_flux = (bb_m_zero < 90) & (nb_m_zero < 90)
    print('mask_neg_flux: {}'.format(len(np.where(~mask_neg_flux)[0])))
    # Mask sources with too high photoz odds
    mask_pz_odds = (odds < 0.98)
    print('mask_pz_odds:  {}'.format(len(np.where(~mask_pz_odds)[0])))
    # Mask sources with proper motion according to Gaia (sigma > 3)
    parallax_relerr = np.abs(cat['parallax_err']) / cat['parallax']
    pmra_relerr = np.abs(cat['pmra_err']) / cat['pmra']
    pmdec_relerr = np.abs(cat['pmdec_err']) / cat['pmdec']
    mask_pmotion = (
          (np.sqrt(parallax_relerr**2 + pmra_relerr**2 + pmdec_relerr**2) > np.sqrt(27))
        | ( np.isnan(parallax_relerr) | np.isnan(pmra_relerr) | np.isnan(pmdec_relerr) )
    )
    print('mask_pmotion:  {}'.format(len(np.where(~mask_pmotion)[0])))
    
    bb_m = bb_m_zero[mask_neg_flux & mask_pz_odds & mask_pmotion]
    nb_m = nb_m_zero[mask_neg_flux & mask_pz_odds & mask_pmotion]
    print('\nN sources: {}'.format(bb_m.shape[0]))

In [ ]:
## Load the catalog
filename = 'csv/minijpas.magAB_photoz_gaia.csv'
cat = load_cat_photoz_gaia(filename)

In [ ]:
p_lya_bbnb(-3, 13, cat)

In [ ]:
cat['pmdec_err']